In [ ]:
from pyspark.sql import SparkSession

# --conf spark.sql.catalog.spark_catalog.uri=thrift://127.0.0.1:9083

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("iceberg_streaming") \
    .config("spark.sql.warehouse.dir", './hive-warehouse') \
    .config('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.type', 'hive') \
    .config('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog') \
    .config('spark.sql.catalog.iceberg.type', 'hadoop') \
    .config('spark.sql.catalog.iceberg.warehouse', './iceberg-warehouse') \
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .getOrCreate()

In [ ]:
spark.sql("show tables in iceberg.db").show(truncate=False)

In [ ]:
path = "./data/activity-data/"
static = spark.read.json(path)
static.createOrReplaceTempView("activityTempView")
print(static.schema)
sql = f"""
    create table iceberg.db.activity
    using iceberg
    partitioned by (device)
    as select * from activityTempView limit 10
    """
spark.sql(sql)

In [ ]:
spark.sql("select * from iceberg.db.activity.files limit 10;").toPandas()